In [ ]:
# syft absolute
import syft as sy
from syft import test_settings

# Only low side server and login as DS

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port="auto",
)

In [ ]:
ds_client = server_low.login(
    email="data_scientist@openmined.org", password="verysecurepassword"
)

# Low side research

In [ ]:
assert len(ds_client.custom_api.api_endpoints()) == 3

In [ ]:
dataset_1 = test_settings.get("dataset_1", default="dataset_1")
dataset_2 = test_settings.get("dataset_2", default="dataset_2")
table_1 = test_settings.get("table_1", default="table_1")
table_2 = test_settings.get("table_2", default="table_2")
table_2_col_id = test_settings.get("table_2_col_id", default="table_id")
table_2_col_score = test_settings.get("table_2_col_score", default="colname")
query_limit_size = test_settings.get("query_limit_size", default=10000)

In [ ]:
result = ds_client.api.services.bigquery.test_query.mock(
    sql_query=f"SELECT * from {dataset_2}.{table_2} limit 10"
)
assert len(result) == 10

In [ ]:
with sy.raises(sy.SyftException, show=True):
    ds_client.api.services.bigquery.test_query.private(
        sql_query=f"SELECT * from {dataset_2}.{table_2} limit 10"
    )

In [ ]:
res = ds_client.api.services.bigquery.schema()
# third party
import pandas as pd

assert isinstance(res.get(), pd.DataFrame)

In [ ]:
FUNC_NAME = "large_sample"
LARGE_SAMPLE_QUERY = f"SELECT * FROM {dataset_2}.{table_2} LIMIT {query_limit_size}"

In [ ]:
mock_res = ds_client.api.services.bigquery.test_query(sql_query=LARGE_SAMPLE_QUERY)

In [ ]:
submission = ds_client.api.services.bigquery.submit_query(
    func_name=FUNC_NAME, query=LARGE_SAMPLE_QUERY
)

In [ ]:
def extract_code_path(response):
    # stdlib
    import re

    pattern = r"client\.code\.(\w+)\(\)"
    match = re.search(pattern, str(response))
    if match:
        extracted_code = match.group(1)
        return extracted_code
    return None

In [ ]:
# why are we randomizing things here?
func_name = extract_code_path(submission)

In [ ]:
api_method = getattr(ds_client.code, func_name, None)
api_method

In [ ]:
# todo: this is very noisy, but it actually passes
with sy.raises(
    sy.SyftException(
        public_message="*Please wait for the admin to allow the execution of this code*"
    ),
    show=True,
):
    result = api_method(blocking=False)

In [ ]:
assert "large_sample" in func_name

In [ ]:
api_method_2 = getattr(ds_client.code, func_name, None)
api_method_2

In [ ]:
with sy.raises(
    sy.SyftException(public_message="*Your code is waiting for approval*"), show=True
):
    result = api_method_2()

In [ ]:
server_low.land()